# Probabilistic method 
This notebook presents an initial approach to understanding the key aspects of probabilistic reserve dimensioning. The main objective is to derive a final probability density function (PDF) that integrates the primary sources of errors, including forecast errors, market deviations, residual noise, asset outages, and transmission line failures.

The first approach focuses on analyzing how different distributions affect the final convoluted distribution. Additionally, a Markov chain model is constructed to predict future imbalances and optimize reserve procurement accordingly.

### Libraries import

In [47]:
import numpy as np
from scipy.stats import norm
from scipy.signal import fftconvolve
import pandas as pd
import plotly.express as px
from scipy.stats import weibull_min

### Random distrubutions

In [48]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import weibull_min
from scipy.signal import fftconvolve

# Step 1: Simulate system imbalances (Normal Distribution)
np.random.seed(42)
num_samples = 10000
system_imbalances = np.random.normal(loc=0, scale=100, size=num_samples)  # Mean 0, Std 100

# Step 1.2: Simulate forced outages (Weibull Distribution for outage durations)
shape_param = 2  # Shape parameter for Weibull distribution
scale_param = 50  # Scale parameter
forced_outages = weibull_min.rvs(shape_param, scale=scale_param, size=num_samples)

# Plot histograms of the two distributions
df_system_imbalances = pd.DataFrame(system_imbalances, columns=["System Imbalances"])
df_forced_outages = pd.DataFrame(forced_outages, columns=["Forced Outages"])

fig1 = px.histogram(df_system_imbalances, x="System Imbalances", nbins=100, title="Histogram of System Imbalances", opacity=0.75)
fig2 = px.histogram(df_forced_outages, x="Forced Outages", nbins=100, title="Histogram of Forced Outages", opacity=0.75)

fig1.show()
fig2.show()

# Step 2: Compute PDFs using histograms
hist1, bin_edges1 = np.histogram(system_imbalances, bins=100, density=True)
hist2, bin_edges2 = np.histogram(forced_outages, bins=100, density=True)

bin_centers1 = (bin_edges1[:-1] + bin_edges1[1:]) / 2
bin_centers2 = (bin_edges2[:-1] + bin_edges2[1:]) / 2

# Plot the PDFs
df_pdf1 = pd.DataFrame({"Bin Centers": bin_centers1, "PDF": hist1})
df_pdf2 = pd.DataFrame({"Bin Centers": bin_centers2, "PDF": hist2})

fig3 = px.line(df_pdf1, x="Bin Centers", y="PDF", title="PDF of System Imbalances", line_shape="spline")
fig4 = px.line(df_pdf2, x="Bin Centers", y="PDF", title="PDF of Forced Outages", line_shape="spline")

fig3.show()
fig4.show()

# Step 3: Convolution of the two PDFs
convoluted_pdf = fftconvolve(hist1, hist2, mode="full")
conv_x_vals = np.linspace(bin_centers1[0] + bin_centers2[0], bin_centers1[-1] + bin_centers2[-1], len(convoluted_pdf))

# Step 4: Compute the quantile for reliability level (99.9%)
reliability_level = 0.999
quantile_value = np.percentile(conv_x_vals, reliability_level * 100)

# Step 5: Visualization of the Convoluted PDF with the Reliability Quantile
fig5 = go.Figure()

fig5.add_trace(go.Scatter(
    x=conv_x_vals, y=convoluted_pdf,
    mode='lines',
    name="Convoluted PDF",
    line=dict(color='red')
))

fig5.add_vline(
    x=quantile_value,
    line=dict(color='blue', dash='dash'),
    annotation_text=f"99.9% Quantile: {quantile_value:.2f} MW",
    annotation_position="top left"
)

fig5.update_layout(
    title="Final Convoluted PDF with 99.9% Reliability Quantile",
    xaxis_title="Power Shortfall (MW)",
    yaxis_title="Density",
    template="plotly_white"
)

fig5.show()

# Print quantile value
print(f"Quantile value for {reliability_level*100}% reliability: {quantile_value:.2f} MW")


Quantile value for 99.9% reliability: 541.08 MW
